In [14]:
import copy

import pytorch_lightning as pl
import torch
import torch.nn as nn
import torchvision

from lightly.data import LightlyDataset, SimCLRCollateFunction, collate
from lightly.loss import NTXentLoss
from lightly.models import ResNetGenerator
from lightly.models.modules.heads import MoCoProjectionHead
from lightly.models.utils import (
    batch_shuffle,
    batch_unshuffle,
    deactivate_requires_grad,
    update_momentum,
)
from lightly.models.modules.heads import SimSiamPredictionHead, SimSiamProjectionHead
 

In [2]:
num_workers = 8
batch_size = 512
memory_bank_size = 4096
seed = 1
max_epochs = 1
input_size=32
pl.seed_everything(seed)
path_to_train = "../Datasets/cifar10/train/"
path_to_test = "../Datasets/cifar10/test/"

Global seed set to 1


In [3]:
# MoCo v2 uses SimCLR augmentations, additionally, disable blur
collate_fn = SimCLRCollateFunction(
    input_size=input_size,
    gaussian_blur=0.0,
)

In [4]:
# Augmentations typically used to train on cifar-10
train_classifier_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.RandomCrop(input_size, padding=4),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            mean=collate.imagenet_normalize["mean"],
            std=collate.imagenet_normalize["std"],
        ),
    ]
)

# No additional augmentations for the test set
test_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize((input_size, input_size)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            mean=collate.imagenet_normalize["mean"],
            std=collate.imagenet_normalize["std"],
        ),
    ]
)
 




In [5]:

# We use the moco augmentations for training moco
dataset_train_simclr = LightlyDataset(input_dir=path_to_train)

# Since we also train a linear classifier on the pre-trained moco model we
# reuse the test augmentations here (MoCo augmentations are very strong and
# usually reduce accuracy of models which are not used for contrastive learning.
# Our linear layer will be trained using cross entropy loss and labels provided
# by the dataset. Therefore we chose light augmentations.)
dataset_train_classifier = LightlyDataset(
    input_dir=path_to_train, transform=train_classifier_transforms
)

dataset_test = LightlyDataset(input_dir=path_to_test, transform=test_transforms)

In [6]:
dataloader_train_simclr = torch.utils.data.DataLoader(
    dataset_train_simclr,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn,
    drop_last=True,
    num_workers=num_workers,
)

dataloader_train_classifier = torch.utils.data.DataLoader(
    dataset_train_classifier,
    batch_size=1,
    shuffle=True,
    drop_last=True,
    num_workers=num_workers,
)

dataloader_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=1,
    shuffle=False,
    drop_last=False,
    num_workers=num_workers,
)

In [7]:
 
class SimSiam(nn.Module):
    def __init__(self, num_ftrs, proj_hidden_dim, pred_hidden_dim, out_dim):
        super().__init__()
        resnet = torchvision.models.resnet18()
        self.backbone = nn.Sequential(*list(resnet.children())[:-1])
        self.projection_head = SimSiamProjectionHead(num_ftrs, proj_hidden_dim, out_dim)
        self.prediction_head = SimSiamPredictionHead(out_dim, pred_hidden_dim, out_dim)

    def forward(self, x):
        # get representations
        f = self.backbone(x).flatten(start_dim=1)
        # get projections
        z = self.projection_head(f)
        # get predictions
        p = self.prediction_head(z)
        # stop gradient
        z = z.detach()
        return z, p


In [8]:
# dimension of the embeddings
num_ftrs = 512
# dimension of the output of the prediction and projection heads
out_dim = proj_hidden_dim = 512
# the prediction head uses a bottleneck architecture
pred_hidden_dim = 128

model = SimSiam( num_ftrs, proj_hidden_dim, pred_hidden_dim, out_dim)

In [ ]:
# SimSiam uses a symmetric negative cosine similarity loss
criterion = NegativeCosineSimilarity()

# scale the learning rate
lr = 0.05 * batch_size / 256
# use SGD with momentum and weight decay
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)




In [ ]:
import math
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

avg_loss = 0.0
avg_output_std = 0.0
for e in range(max_epochs):
    for (x0, x1), _, _ in dataloader_train_simclr:
        # move images to the gpu
        x0 = x0.to(device)
        x1 = x1.to(device)

        # run the model on both transforms of the images
        # we get projections (z0 and z1) and
        # predictions (p0 and p1) as output
        z0, p0 = model(x0)
        z1, p1 = model(x1)

        # apply the symmetric negative cosine similarity
        # and run backpropagation
        loss = 0.5 * (criterion(z0, p1) + criterion(z1, p0))
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

        # calculate the per-dimension standard deviation of the outputs
        # we can use this later to check whether the embeddings are collapsing
        output = p0.detach()
        output = torch.nn.functional.normalize(output, dim=1)

        output_std = torch.std(output, 0)
        output_std = output_std.mean()

        # use moving averages to track the loss and standard deviation
        w = 0.9
        avg_loss = w * avg_loss + (1 - w) * loss.item()
        avg_output_std = w * avg_output_std + (1 - w) * output_std.item()

    # the level of collapse is large if the standard deviation of the l2
    # normalized output is much smaller than 1 / sqrt(dim)
    collapse_level = max(0.0, 1 - math.sqrt(out_dim) * avg_output_std)
    # print intermediate results
    print(
        f"[Epoch {e:3d}] "
        f"Loss = {avg_loss:.2f} | "
        f"Collapse Level: {collapse_level:.2f} / 1.00"
    )

In [ ]:
from sklearn import svm
import numpy as np
import torch
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.eval()
model.to(device)

# Initialize features and labels as empty numpy arrays
X_train = np.array([])
y_train = np.array([])

# Wrap your dataloader with tqdm for a progress bar
for image, target, fname in tqdm(dataloader_train_classifier):
    with torch.no_grad():
        # Forward pass to extract features
        # Note: You might need to modify this depending on the output of your SimCLR model
        feature = model.backbone(image.to(device)).cpu().numpy().flatten()
        target = target.cpu().numpy().flatten()

        # If features and labels are empty, assign the first feature and label
        # Else, stack the new feature and label as a new row

        if X_train.size == 0 and y_train.size == 0:
            X_train = feature
            y_train = target
        else:
            X_train = np.vstack((X_train, feature))
            y_train = np.hstack((y_train, target))

In [ ]:
# Train the SVM classifier
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train, y_train)


X_test_predicted = clf.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test, X_test_predicted)

In [ ]:
X_test.shape

In [17]:
from sklearn import svm
from sklearn.metrics import accuracy_score
import numpy as np
import tqdm
# Initialize features and labels as empty numpy arrays
X_test = np.array([])
y_test = np.array([])
fname_test=[]
for image, target, fname in tqdm(dataloader_test):
    with torch.no_grad():
        
        feature = model.backbone(image.to(device)).cpu().numpy().flatten()
        target = target.cpu().numpy().flatten()

    if X_test.size == 0 and y_test.size == 0:
        X_test = feature
        y_test = target
    else:
        X_test = np.vstack((X_test, feature))
        y_test = np.hstack((y_test, target))
    fname_test.append(fname)


TypeError: 'module' object is not callable

In [27]:

# Predict labels for test data
X_test_predicted = clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, X_test_predicted)

print(f"Model accuracy: {accuracy}")


Model accuracy: 0.04296100462656973


In [11]:
root_path = '../Datasets/cifar10/test/'
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

random_indices = random.sample(range(X_test.shape[0]), 2)
for idx in random_indices:
    distances, indices = nbrs.kneighbors(X_test[idx].reshape(1, -1))
    fig, axs = plt.subplots(3, 3, figsize=(10, 10))
    fig.suptitle('Nearest neighbors for: '+str(fname_test[idx][0]), fontsize=16)
    example_label = y_test[idx]
    
    # Iterate over the neighbors
    for i, index in enumerate(indices[0]):
        image_path = root_path + fname_test[index][0]  # Extract the filename from the tuple and prepend the root path
        img = mpimg.imread(image_path)
        label = y_test[index]
        
        # Choose label color based on match with example
        color = 'red' if label != example_label else 'black'
        
        # Plot image and label
        axs[i//3, i%3].imshow(img)
        axs[i//3, i%3].set_title('Label: '+str(label), color=color)
        axs[i//3, i%3].axis('off')

    plt.tight_layout()
    plt.show()


NameError: name 'X_test' is not defined